<a href="https://colab.research.google.com/github/dgudlek/password-cracking-velocity/blob/main/hashcat_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install hashcat &> /dev/null

In [2]:
# Change the root password to 'zzzz' and copy the hash.
!echo root:zzzz | chpasswd
!cat /etc/shadow | grep -E '^root:' | cut -f2 -d: > stolen_hash.txt
!cat stolen_hash.txt

$6$UG2kTqI9$nh5GUoVdKmbNvpSUerNMtchtVt2DNWLcUx0RZ0P94tQWoWfWxMLCMDOhsV6KI/PO9qjUKnNGhqkMLzzZpzYXF/


In [3]:
%%bash
# Installs opencl runtime for CPU
# https://software.intel.com/en-us/articles/opencl-drivers#latest_CPU_runtime
PACKAGE_URL=http://registrationcenter-download.intel.com/akdlm/irc_nas/12556/opencl_runtime_16.1.2_x64_rh_6.4.0.37.tgz
PACKAGE_NAME=opencl_runtime_16.1.2_x64_rh_6.4.0.37
wget -q ${PACKAGE_URL} -O /tmp/opencl_runtime.tgz
tar -xzf /tmp/opencl_runtime.tgz -C /tmp
sed 's/decline/accept/g' -i /tmp/${PACKAGE_NAME}/silent.cfg
apt-get install -yq cpio
/tmp/${PACKAGE_NAME}/install.sh -s /tmp/${PACKAGE_NAME}/silent.cfg

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  libarchive1
The following NEW packages will be installed:
  cpio
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 86.4 kB of archives.
After this operation, 356 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 cpio amd64 2.12+dfsg-6ubuntu0.18.04.4 [86.4 kB]
Fetched 86.4 kB in 0s (211 kB/s)
Selecting previously unselected package cpio.
(Reading database ... 156141 files and directories currently installed.)
Preparing to unpack .../cpio_2.12+dfsg-6ubuntu0.18.04.4_amd64.deb ...
Unpacking cpio (2.12+dfsg-6ubuntu0.18.04.4) ...
Setting up cpio (2.12+dfsg-6ubuntu0.18.04.4) ...
update-alternatives: using /bin/mt-gnu to provide /bin/mt (mt) in auto mode
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Missing optional prerequisites
-- Unsupported OS
-------------------------------------------------------

In [4]:
!hashcat --opencl-info

hashcat (v4.0.1) starting...

OpenCL Info:

Platform ID #1
  Vendor  : Intel(R) Corporation
  Name    : Intel(R) OpenCL
  Version : OpenCL 2.0 LINUX

  Device ID #1
    Type           : CPU
    Vendor ID      : 8
    Vendor         : Intel(R) Corporation
    Name           : Intel(R) Xeon(R) CPU @ 2.20GHz
    Version        : OpenCL 2.0 (Build 37)
    Processor(s)   : 2
    Clock          : 2200
    Memory         : 3247/12991 MB allocatable
    OpenCL Version : OpenCL C 2.0 
    Driver Version : 1.2.0.37



In [5]:
!hashcat --potfile-disable --markov-disable -m 1800 -a 3 stolen_hash.txt ?l?l?l?l

hashcat (v4.0.1) starting...

OpenCL Platform #1: Intel(R) Corporation
* Device #1: Intel(R) Xeon(R) CPU @ 2.20GHz, 3247/12991 MB allocatable, 2MCU

Hashes: 1 digests; 1 unique digests, 1 unique salts
Bitmaps: 16 bits, 65536 entries, 0x0000ffff mask, 262144 bytes, 5/13 rotates

Applicable optimizers:
* Zero-Byte
* Single-Hash
* Single-Salt
* Brute-Force
* Uses-64-Bit

Password length minimum: 0
Password length maximum: 256

ATTENTION! Pure (unoptimized) OpenCL kernels selected.
This enables cracking passwords and salts > length 32 but for the price of drastical reduced performance.
If you want to switch to optimized OpenCL kernels, append -O to your commandline.

Watchdog: Hardware monitoring interface not found on your system.
Watchdog: Temperature abort trigger disabled.
Watchdog: Temperature retain trigger disabled.

* Device #1: build_opts '-I /usr/share/hashcat/OpenCL -D VENDOR_ID=8 -D CUDA_ARCH=0 -D AMD_ROCM=0 -D VECT_SIZE=4 -D DEVICE_TYPE=2 -D DGST_R0=0 -D DGST_R1=1 -D DGST_R2=2